In [17]:
import os
import pandas as pd
from datetime import date
from pony.orm import *

db = Database()

In [18]:
!pip install pony

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Define how to map tables to objects

In [19]:
class Station(db.Entity):
    stations_id = PrimaryKey(int, auto=False)
    von_datum = Optional(date)
    bis_datum = Optional(date)
    stationshoehe = Optional(int)
    geoBreite = Optional(float)
    geoLaenge = Optional(float)
    stationsname = Required(str)
    bundesland = Optional(str)
    measurements = Set('Measurement')


class Measurement(db.Entity):
    mess_datum = Required(int)      # have to set it to date later. sqlite has some problems with this
    stations_id = Required(Station)
    qn_3 = Optional(int)  # quality level of next columns
    fx = Optional(float)
    fm = Optional(float)
    qn_4 = Optional(int)
    rsk = Optional(float)
    rskf = Optional(float)
    sdk = Optional(float)
    shk_tag = Optional(float)
    nm = Optional(float)
    vpm = Optional(float)
    pm = Optional(float)
    tmk = Optional(float)
    upm = Optional(float)
    txk = Optional(float)
    tnk = Optional(float)
    tgk = Optional(float)
    PrimaryKey(mess_datum, stations_id)

In [20]:
db.bind(provider='sqlite', filename=os.path.abspath('./database.db'))

In [21]:
db.generate_mapping()

In [6]:
# if we want to generate the database tables using this
# db.generate_mapping(create_tables=True)

We already have the database set up by sqlalchemy, so we only want to query

In [22]:
stations = select(s for s in Station)

In [23]:
len(stations)

2

In [24]:
for s in stations:
    print(s.stations_id, s.stationsname)

1 Aach
3 Aachen


Pulling the whole measurements table into a list of objects and calculating the length is really slow

In [25]:
measurements = select(m for m in Measurement)

In [26]:
len(measurements)

55788

Instead just count the measurements in the table

In [27]:
count(m for m in Measurement)

55788

Filter for the measurements in Aach and only count those

In [28]:
count(s.measurements for s in Station if s.stationsname == 'Aach')

17348

We can read whole tables into pandas without sqlalchemy or pony

In [29]:
pd.read_sql_table('measurement', 'sqlite:///database.db')

,mess_datum,stations_id,qn_3,fx,fm,qn_4,rsk,rskf,sdk,shk_tag,nm,vpm,pm,tmk,upm,txk,tnk,tgk
0,18910101,3,NaN,NaN,NaN,5,0.0,0.0,NaN,NaN,0.0,4.3,NaN,-3.6,88.0,0.5,-5.9,NaN
1,18910102,3,NaN,NaN,NaN,5,0.0,0.0,NaN,NaN,2.7,4.1,NaN,-2.8,84.0,0.0,-5.8,NaN
2,18910103,3,NaN,NaN,NaN,5,2.5,1.0,NaN,NaN,3.7,3.9,NaN,-0.2,69.0,2.1,-6.2,NaN
3,18910104,3,NaN,NaN,NaN,5,8.2,1.0,NaN,NaN,8.0,6.4,NaN,1.8,90.0,3.7,0.6,NaN
4,18910105,3,NaN,NaN,NaN,5,1.9,1.0,NaN,NaN,7.7,4.7,NaN,-2.5,87.0,1.5,-4.2,NaN
5,18910106,3,NaN,NaN,NaN,5,2.5,1.0,NaN,NaN,8.0,3.5,NaN,-5.8,88.0,-4.0,-6.9,NaN
6,18910107,3,NaN,NaN,NaN,5,2.9,1.0,NaN,NaN,8.0,3.6,NaN,-5.8,94.0,-5.0,-7.7,NaN
7,18910108,3,NaN,NaN,NaN,5,0.0,0.0,NaN,NaN,6.7,3.7,NaN,-5.0,89.0,-3.5,-5.7,NaN
8,18910109,3,NaN,NaN,NaN,5,0.0,0.0,NaN,NaN,6.7,2.9,NaN,-7.8,87.0,-4.7,-9.2,NaN
9,18910110,3,NaN,NaN,NaN,5,0.0,0.0,NaN,NaN,3.7,2.1,NaN,-11.4,85.0,-7.6,-12.7,NaN


But we can also read any query into a pandas dataframe by giving it the SQL query as text generated by pony

In [31]:
qu = select(s.measurements for s in Station if s.stationsname == 'Aach')

In [32]:
pd.read_sql(qu.get_sql(), 'sqlite:///database.db')

,mess_datum,stations_id,qn_3,fx,fm,qn_4,rsk,rskf,sdk,shk_tag,nm,vpm,pm,tmk,upm,txk,tnk,tgk
0,19370101,1,None,None,None,5,0.0,0.0,None,0.0,6.3,NaN,None,-0.5,NaN,2.5,-1.6,NaN
1,19370102,1,None,None,None,5,0.0,0.0,None,0.0,3.0,NaN,None,0.3,NaN,5.0,-4.0,NaN
2,19370103,1,None,None,None,5,0.0,0.0,None,0.0,4.3,NaN,None,3.2,NaN,5.0,-0.2,NaN
3,19370104,1,None,None,None,5,0.0,0.0,None,0.0,8.0,NaN,None,0.2,NaN,3.8,-0.2,NaN
4,19370105,1,None,None,None,5,0.0,0.0,None,0.0,8.0,NaN,None,1.4,NaN,4.5,-0.7,NaN
5,19370106,1,None,None,None,5,5.2,7.0,None,0.0,6.0,NaN,None,0.2,NaN,2.0,-2.4,NaN
6,19370107,1,None,None,None,5,3.6,1.0,None,0.0,6.3,NaN,None,5.4,NaN,8.2,0.6,NaN
7,19370108,1,None,None,None,5,0.8,1.0,None,0.0,4.7,NaN,None,2.1,NaN,4.0,1.4,NaN
8,19370109,1,None,None,None,5,0.0,0.0,None,0.0,0.7,NaN,None,-1.6,NaN,1.6,-3.4,NaN
9,19370110,1,None,None,None,5,0.0,0.0,None,0.0,0.0,NaN,None,-3.0,NaN,1.4,-7.5,NaN


In [33]:
quc = select(count(s.measurements) for s in Station if s.stationsname == 'Aach')
pd.read_sql(quc.get_sql(), 'sqlite:///database.db')

,"COUNT(""measurement"".""ROWID"")"
0,17348


In [34]:
old_measurements = select(m for m in Measurement if m.stations_id.stationsname == 'Aachen' and m.mess_datum < 19000101)

In [39]:
for m in old_measurements[:5]:
    print(m)

Measurement[18910101,Station[3]]
Measurement[18910102,Station[3]]
Measurement[18910103,Station[3]]
Measurement[18910104,Station[3]]
Measurement[18910105,Station[3]]


It looks a bit weird when we reference the station of a measurement as stations_id. Could redesign that if we use this pony

In [37]:
old_measurements.

<bound method count of <pony.orm.core.Query object at 0x7faceff9beb8>>